Implementing Iteration
======================



## Agenda



1.  Review: Iteration
2.  Details: *iterables*, *iterators*, `iter`, and `next`
3.  Implementing iterators with classes
4.  Implementing iterators with *generators* and `yield`



## 1.  Review: Iteration



*Iteration* simply refers to the process of accessing &#x2014; one by one &#x2014;
the items stored in some container. The order of the items, and whether
or not the iteration is comprehensive, depends on the container.

In Python, we typically perform iteration using the `for` loop.



In [1]:
# e.g., iterating over a list
l = [2**x for x in range(10)]
for n in l:
    print(n)

1
2
4
8
16
32
64
128
256
512


In [2]:
# e.g., iterating over the key-value pairs in a dictionary
d = {x:2**x for x in range(10)}
for k,v in d.items():
    print(k, '=>', v)

0 => 1
1 => 2
2 => 4
3 => 8
4 => 16
5 => 32
6 => 64
7 => 128
8 => 256
9 => 512


## 2.  Details: *iterables*, *iterators*, `iter`, and `next`



We can iterate over anything that is *iterable*. Intuitively, if
something can be used as the source of items in a `for` loop, it is
iterable.

But how does a `for` loop really work? (Review time!)
1. Have thing you want to iterate over
2. Inside a while loop...
- have a try/except statement 
- if there's an exception, break out of the loop



In [3]:
l = [2**x for x in range(10)]

In [18]:
it = iter(l)

In [6]:
it is iter(it)

True

In [29]:
next(it)

StopIteration: 

In [30]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [31]:
it = l.__iter__()
while True:
    try:
        x = it.__next__()
        print(x)
    except StopIteration:
        break

1
2
4
8
16
32
64
128
256
512


## 3.  Implementing iterators with classes



In [51]:
class MyIterator: # tries to behave like a `range`
    def __init__(self, max):
        self.max = max
        self.curr = 0 # current value

    # the following methods are required for iterator objects

    def __next__(self):
        if self.curr < self.max:
            rval = self.curr
            self.curr += 1
            return rval
        else:
            raise StopIteration()

    def __iter__(self):
        return self

In [37]:
it = MyIterator(10)

In [48]:
next(it)

StopIteration: 

In [49]:
it = MyIterator(10)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

0
1
2
3
4
5
6
7
8
9


In [58]:
it = MyIterator(10)
for i in it:
    print(i)

0
1
2
3
4
5
6
7
8
9


For a container type, we need to implement an `__iter__` method that
returns an iterator.



In [88]:
class ArrayList:
    
    def __init__(self):
        self.data = []

    def append(self, val):
        self.data.append(None)
        self.data[len(self.data)-1] = val

    def __iter__(self):
        class ArrayListIterator:
            def __init__(self, data):
                self.curr_idx = 0
        
        # methods required for iterator objects
        
            def __next__(self):
                if self.curr_idx < len(self.data):
                    rval = self.data[self.curr_idx]
                    self.curr_idx += 1
                    return rval
                else:
                    raise StopIteration()
        
            def __iter__(self):
                return self
        
        return ArrayListIterator(self.data) 

In [103]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [104]:
it = iter(l)

In [105]:
type(it)

__main__.ArrayList.__iter__.<locals>.ArrayListIterator

In [113]:
next(it)

128

In [87]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


## 4.  Implementing iterators with generators and `yield`



What's a "generator"?



In [114]:
l = [2**x for x in range(10)]

In [115]:
l

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]

In [116]:
type(l)

list

In [170]:
g = (2**x for x in range(100))

In [164]:
type(g)

generator

In [171]:
for x in g:
    print(x)

1
2
4
8
16
32
64
128
256
512
1024
2048
4096
8192
16384
32768
65536
131072
262144
524288
1048576
2097152
4194304
8388608
16777216
33554432
67108864
134217728
268435456
536870912
1073741824
2147483648
4294967296
8589934592
17179869184
34359738368
68719476736
137438953472
274877906944
549755813888
1099511627776
2199023255552
4398046511104
8796093022208
17592186044416
35184372088832
70368744177664
140737488355328
281474976710656
562949953421312
1125899906842624
2251799813685248
4503599627370496
9007199254740992
18014398509481984
36028797018963968
72057594037927936
144115188075855872
288230376151711744
576460752303423488
1152921504606846976
2305843009213693952
4611686018427387904
9223372036854775808
18446744073709551616
36893488147419103232
73786976294838206464
147573952589676412928
295147905179352825856
590295810358705651712
1180591620717411303424
2361183241434822606848
4722366482869645213696
9444732965739290427392
18889465931478580854784
37778931862957161709568
75557863725914323419136
151

In [124]:
iter(g) is g # generators behave like an iterator

True

In [158]:
next(g)

512

In [174]:
g = (2**x for x in range(1_000_000))

TypeError: 'generator' object is not subscriptable

Generators represent a computation that hasn't yet happened. They aren't indexable.
* They are incredibly useful when all we want to do is iterate through values